In [10]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input
import tensorflow.lite as tflite
from io import BytesIO
from urllib import request

from PIL import Image

In [6]:
model = keras.models.load_model('bees-wasps.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('bees-wasps.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

2023-11-29 17:12:03.715531: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-29 17:12:03.716017: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


INFO:tensorflow:Assets written to: /tmp/tmpxplejd5t/assets


INFO:tensorflow:Assets written to: /tmp/tmpxplejd5t/assets
2023-11-29 17:12:07.383152: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-29 17:12:07.383200: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-11-29 17:12:07.383833: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpxplejd5t
2023-11-29 17:12:07.384718: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-11-29 17:12:07.384745: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpxplejd5t
2023-11-29 17:12:07.387383: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2023-11-29 17:12:07.388033: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-11-29 17:12:07.631562: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

In [7]:
!ls -la

total 131476
drwxr-xr-x 2 andy andy     4096 Nov 29 17:12 .
drwxr-xr-x 6 andy andy     4096 Nov 29 17:07 ..
-rw-r--r-- 1 andy andy 89753864 Nov 17 23:51 bees-wasps.h5
-rw-r--r-- 1 andy andy 44866200 Nov 29 17:12 bees-wasps.tflite
-rw-r--r-- 1 andy andy        0 Nov 29 17:07 notebook.ipynb


In [12]:
interpreter = tflite.Interpreter(model_path='bees-wasps.tflite')
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']
print(f"input_index={input_index}, output_index={output_index}")

input_index=0, output_index=13


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [13]:
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [20]:
url = 'https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg'
img = download_image(url)
prepared_image = prepare_image(img, (150, 150))

In [42]:
X = np.array(prepared_image) * (1. / 255)
X = np.array([X], dtype="float32")

In [43]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [46]:
preds[0, 0]

0.6592137